# 1

Przygotować trzy przykłady pokazujące następujące anomalie: odczyty „brudnych danych”, niepowtarzalność odczytów, odczyty fantomów. Sprawdzić eksperymentalnie, w ramach których poziomów izolacji transakcji wymienione anomalie występują.

In [111]:
SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED
-- SET TRANSACTION ISOLATION LEVEL READ COMMITTED;
-- SET TRANSACTION ISOLATION LEVEL REPEATABLE READ;
-- SET TRANSACTION ISOLATION LEVEL SNAPSHOT;
-- SET TRANSACTION ISOLATION LEVEL SERIALIZABLE;

Commands completed successfully.

Total execution time: 00:00:00.058

In [5]:
DROP TABLE IF EXISTS Test_zad1
GO
    
CREATE TABLE Test_zad1
(
    ID INT PRIMARY KEY,
    Book VARCHAR(100), 
    Quantity INT
);
GO

INSERT INTO Test_zad1(ID, Book, Quantity)
VALUES (1, 'Harry Potter and the Philosopher Stone', '3'),
       (2, 'Da Vinci Code', '5'),
       (3, 'Harry Potter and the Chamber of Secrets', 2),
       (4, 'One Day', 3),
       (5, 'Labyrinth', 9),
       (6, 'Lord of the Rings', 1),
       (7, 'Short History of Tractors in Ukrainian', 5)

Commands completed successfully.

Commands completed successfully.

(7 rows affected)

Total execution time: 00:00:00.181

In [105]:
SELECT * FROM Test_zad1

Query was canceled by user

: Query failed: The operation was canceled.

Query was canceled by user

Total execution time: 00:00:02.607

In [114]:
SELECT * FROM sys.dm_tran_locks

(11 rows affected)

Total execution time: 00:00:00.072

resource_type,resource_subtype,resource_database_id,resource_description,resource_associated_entity_id,resource_lock_partition,request_mode,request_type,request_status,request_reference_count,request_lifetime,request_session_id,request_exec_context_id,request_request_id,request_owner_type,request_owner_id,request_owner_guid,request_owner_lockspace_id,lock_owner_address
DATABASE,,6,,0,0,S,LOCK,GRANT,1,0,78,0,0,SHARED_TRANSACTION_WORKSPACE,0,00000000-0000-0000-0000-000000000000,0x000001D62DC72A50:0:0,0xA0EBDB8A37BCCB00
DATABASE,,6,,0,0,S,LOCK,GRANT,1,0,74,0,0,SHARED_TRANSACTION_WORKSPACE,0,00000000-0000-0000-0000-000000000000,0x000001D62DC73E50:0:0,0x04EDE4B5F96F9883
DATABASE,,6,,0,0,S,LOCK,GRANT,1,0,81,0,0,SHARED_TRANSACTION_WORKSPACE,0,00000000-0000-0000-0000-000000000000,0x000001D63A3129C0:0:0,0x27AAEA13F010A2CD
DATABASE,,6,,0,0,S,LOCK,GRANT,1,0,80,0,0,SHARED_TRANSACTION_WORKSPACE,0,00000000-0000-0000-0000-000000000000,0x000001D62DC72370:0:0,0x64BA9015E8D4B8C2
DATABASE,,6,,0,0,S,LOCK,GRANT,1,0,77,0,0,SHARED_TRANSACTION_WORKSPACE,0,00000000-0000-0000-0000-000000000000,0x000001D62DC729C0:0:0,0x2FA9F748167990B1
DATABASE,,6,,0,0,S,LOCK,GRANT,1,0,76,0,0,SHARED_TRANSACTION_WORKSPACE,0,00000000-0000-0000-0000-000000000000,0x000001D62DC73130:0:0,0x6EEFC37F7A17981C
DATABASE,,6,,0,0,S,LOCK,GRANT,1,0,75,0,0,SHARED_TRANSACTION_WORKSPACE,0,00000000-0000-0000-0000-000000000000,0x000001D62DC72AE0:0:0,0x6355323BEB29905B
DATABASE,,6,,0,0,S,LOCK,GRANT,1,0,57,0,0,SHARED_TRANSACTION_WORKSPACE,0,00000000-0000-0000-0000-000000000000,0x000001D62DC73AE0:0:0,0xEF67347147A26200
DATABASE,,6,,0,0,S,LOCK,GRANT,1,0,70,0,0,SHARED_TRANSACTION_WORKSPACE,0,00000000-0000-0000-0000-000000000000,0x000001D63A313930:0:0,0x2045483DF3ED8A54
DATABASE,,6,,0,0,S,LOCK,GRANT,1,0,64,0,0,SHARED_TRANSACTION_WORKSPACE,0,00000000-0000-0000-0000-000000000000,0x000001D62DC732E0:0:0,0xA31DE3777BD77739


In [4]:
COMMIT

: Msg 3902, Level 16, State 1, Line 1
The COMMIT TRANSACTION request has no corresponding BEGIN TRANSACTION.

Total execution time: 00:00:00.052

## Odczyty "brudnych danych"

uncommitted dependency, (pobranie częściowo zmienionych ale nie zakomitowanych zmian)

In [89]:
BEGIN TRANSACTION
UPDATE Test_zad1
SET Quantity = 100 WHERE ID = 3
WAITFOR DELAY '00:00:25'
ROLLBACK TRANSACTION

: Msg 1222, Level 16, State 51, Line 2
Lock request time out period exceeded.

The statement has been terminated.

Query was canceled by user

Total execution time: 00:00:13.707

Dzieje się na poziomie izolacji READ UNCOMITTED, READ COMMITED naprawia

## Niepowtarzalność odczytów


Odczytujemy ten sam zbiór a dostajemy różne wyniki w różnych momentach

In [119]:
BEGIN TRANSACTION
UPDATE 
    Test_zad1
SET 
    Quantity = 100
WHERE 
    ID = 1
COMMIT

(1 row affected)

Total execution time: 00:00:00.068

Dzieje się na poziomie izolacji READ UNCOMITTED i READ COMMITED

## Odczyty fantomów


In [6]:
BEGIN TRANSACTION
INSERT INTO Test_zad1(ID, Book, Quantity)
VALUES (9, 'The Hobbit', 123)
COMMIT

(1 row affected)

Total execution time: 00:00:00.067